In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from IPython.display import display, Markdown
import streamlit as st

# Constants for the model and API
MODEL = "llama3.2"
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}


In [2]:
# Data structure for holding parsed webpage data
class Website:
    def __init__(self, title, text):
        self.title = title
        self.text = text

In [3]:
# Prompt instructions to the model
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary, ignoring text that might be navigation related. "
    "Respond in markdown."
)

def user_prompt_for(website):
    return (
        f"You are looking at a website titled {website.title}\n"
        "The contents of this website is as follows; please provide a short summary of this website in markdown. "
        "If it includes news or announcements, then summarize these too.\n\n"
        + website.text
    )

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


In [4]:
def summarize(url):
    # Start browser
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)  # Fast but enough for most JS to render

    # Parse the page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()  # Free up browser ASAP

    # Remove noisy HTML elements
    for tag in soup(["img", "style", "script", "nav", "footer", "header", "button", "input", "svg", "a"]):
        tag.decompose()
    for unwanted in soup.select(".navbar, .menu, .sidebar, .ad-banner"):
        unwanted.decompose()

    # Extract content
    title = soup.title.string if soup.title else "No Title"
    text = soup.get_text(separator="\n", strip=True)

    website = Website(title, text)
    payload = {
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }

    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']


In [6]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [7]:
if __name__ == "__main__":
    url = "https://www.tutorialspoint.com/machine_learning/index.htm"
    display_summary(url)


**Summary**
===============

### Overview

The Machine Learning Tutorial provides an in-depth understanding of machine learning concepts, including different types of machine learning algorithms, applications, libraries used in ML, and real-life examples.

### Key Topics

*   **What is Machine Learning?**: A branch of artificial intelligence that works on algorithm developments and statistical models to allow computers to learn from data and make predictions or decisions without being explicitly programmed.
*   **Machine Learning Process**: The tutorial explains the common working process of machine learning, which includes stages such as data collection, data pre-processing, choosing the right model, training the model, evaluating the model, hyperparameter tuning, and deployment.
*   **Types of Machine Learning**: Supervised, unsupervised, semi-supervised, and reinforcement learning are discussed.
*   **Common Machine Learning Algorithms**: Neural networks, linear regression, logistic regression, clustering, decision trees, random forests, and importance of machine learning.
*   **Applications of Machine Learning**: Speech recognition, customer service, computer vision, recommendation engines, robotic process automation, automated stock trading, and fraud detection.

### Recommendations

The tutorial is recommended for those who want to learn about the basics and advances of machine learning. It requires a basic understanding of technical aspects of machine learning, data processing, and artificial intelligence basics.

### Key Takeaways

*   Machine learning is a branch of artificial intelligence that allows computers to learn from data and make predictions or decisions without being explicitly programmed.
*   The machine learning process includes stages such as data collection, data pre-processing, choosing the right model, training the model, evaluating the model, hyperparameter tuning, and deployment.
*   Supervised, unsupervised, semi-supervised, and reinforcement learning are types of machine learning.
*   Machine learning is used in various fields, including speech recognition, customer service, computer vision, recommendation engines, robotic process automation, automated stock trading, and fraud detection.